In [6]:
# Load env data
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
# Pinecone API Key
api_key = "pcsk_7NtNhd_4J3cAYKvZvBaEDDougGjEtvpLTmoVJZfjsaGRwZFesQqafaRNWeejxDV9W7nxPw"

In [3]:
from pinecone import Pinecone, ServerlessSpec

# Initialize the pinecone client
pc = Pinecone(api_key=api_key)

# Create the index
index_name = "hybrid-search-langchain-pinecone"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name = index_name,
        dimension = 384,  ### dimension of dense vector
        metric="dotproduct",  ### sparse values supported only for dotproduct
        spec=ServerlessSpec(
            cloud="aws", 
            region="us-east-1"
        )
    )

In [4]:
index = pc.Index(index_name)
index

In [8]:
# Vector embedding
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [9]:
print(embeddings)

model_name='all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


In [11]:
# Create pinecone encoder
from pinecone_text.sparse import BM25Encoder

encoder = BM25Encoder.default()
encoder

In [17]:
sentences = [
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orleans"
]

# TF-IDF values on the sentence
encoder.fit(sentences)

# Store the values into a json file
encoder.dump("encoded_values.json")

# Load encoded object
encoder = BM25Encoder().load("encoded_values.json")

100%|██████████| 3/3 [00:00<00:00, 3598.20it/s]


In [20]:
# Create retriever
from langchain_community.retrievers import PineconeHybridSearchRetriever

retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=encoder, index=index)
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x7c5ff5e11940>, index=<pinecone.data.index.Index object at 0x7c615eb521b0>)

In [21]:
retriever.add_texts([
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orleans"
    ]
)

100%|██████████| 1/1 [00:02<00:00,  2.41s/it]


In [22]:
retriever.invoke("What city did I visited first?")

[Document(metadata={'score': 0.27135998}, page_content='In 2021, I visited New Orleans'),
 Document(metadata={'score': 0.259855092}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.237034142}, page_content='In 2023, I visited Paris')]